In [39]:
import numpy as np
import random
import math
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

# Parameters
POP_SIZE = 100           
NUM_BITS = 10            
GENERATIONS = 100       
DEFAULT_MUT_RATE = 0.1   
DEFAULT_CROSS_RATE = 0.8 


def decode(binary_string, lower_bound=0, upper_bound=1):
    decimal_value = int("".join(map(str, binary_string)), 2)
    max_decimal = 2**NUM_BITS - 1
    return lower_bound + (decimal_value / max_decimal) * (upper_bound - lower_bound)


def fitness_x_squared(individual):
    x = decode(individual, lower_bound=1, upper_bound=50)  
    return x**2  

def fitness_m1(individual):
    x = decode(individual)  
    return math.sin(5 * math.pi * x)**6

def fitness_m4(individual):
    x = decode(individual)  #
    return math.exp(-2 * (math.log(2)) * ((x - 0.08) / 0.854)**2) * math.sin(5 * math.pi * (x**0.75 - 0.05))**6


def initialize_pop():
    return [[random.randint(0, 1) for _ in range(NUM_BITS)] for _ in range(POP_SIZE)]

# Selection: Keep the top 50% of the population
def selection(population, fitness_function):
    fitness_values = [(ind, fitness_function(ind)) for ind in population]
    sorted_population = sorted(fitness_values, key=lambda x: x[1], reverse=True)
    return [x[0] for x in sorted_population[:POP_SIZE // 2]]

#Single-point crossover
def crossover(parents, crossover_rate=DEFAULT_CROSS_RATE):
    offspring = []
    for _ in range(POP_SIZE):
        parent1 = random.choice(parents)
        parent2 = random.choice(parents)
        if random.random() < crossover_rate:
            crossover_point = random.randint(1, NUM_BITS - 1)
            child = parent1[:crossover_point] + parent2[crossover_point:]
        else:
            child = parent1
        offspring.append(child)
    return offspring


def mutate(offspring, mutation_rate=DEFAULT_MUT_RATE):
    mutated_offspring = []
    for individual in offspring:
        for i in range(len(individual)):
            if random.random() < mutation_rate:
                individual[i] = 1 - individual[i]  
        mutated_offspring.append(individual)
    return mutated_offspring


# Fitness Sharing
def sharing_function(distance, sigma_share, alpha=1):
    """
    Penalizes fitness based on similarity (distance) between individuals.
    """
    if distance < sigma_share:
        return 1 - (distance / sigma_share) ** alpha
    return 0

def apply_fitness_sharing(population, fitness_function, sigma_share=0.1):
    """
    Apply fitness sharing to modify fitness values of the population.
    """
    shared_fitness_values = []
    for i, ind_i in enumerate(population):
        niche_count = 0
        for j, ind_j in enumerate(population):
            if i != j:  # Don't compare an individual to itself
                distance = abs(decode(ind_i) - decode(ind_j))  # Phenotypic distance
                niche_count += sharing_function(distance, sigma_share)
        
        original_fitness = fitness_function(ind_i)
        shared_fitness = original_fitness / (1 + niche_count)  # Penalize fitness
        shared_fitness_values.append(shared_fitness)
    return shared_fitness_values

# Crowding
def crowding_replacement(parents, offspring, fitness_function):
    """
    Replace parents with offspring based on similarity and fitness.
    """
    new_population = []
    for child in offspring:
        parent = min(parents, key=lambda p: abs(decode(p) - decode(child)))  # Most similar parent
        if fitness_function(child) > fitness_function(parent):
            new_population.append(child)  # Replace parent with child
        else:
            new_population.append(parent)  # Keep parent
    return new_population

# Main Genetic Algorithm
def genetic_algorithm(fitness_function, lower_bound=0, upper_bound=1, mutation_rate=DEFAULT_MUT_RATE, crossover_rate=DEFAULT_CROSS_RATE, generations=GENERATIONS, niching_method=None):
    population = initialize_pop()
    initial_population = population[:]

    Fmax_list, Favg_list, Fmin_list = [], [], []

    for generation in range(generations):
        # Apply fitness sharing if selected
        if niching_method == "sharing":
            fitness_values = apply_fitness_sharing(population, fitness_function, sigma_share=0.1)
        else:
            fitness_values = [fitness_function(ind) for ind in population]

        Fmax = max(fitness_values)
        Favg = sum(fitness_values) / len(fitness_values)
        Fmin = min(fitness_values)
        Fmax_list.append(Fmax)
        Favg_list.append(Favg)
        Fmin_list.append(Fmin)

        parents = selection(population, fitness_function)
        offspring = crossover(parents, crossover_rate)
        offspring = mutate(offspring, mutation_rate)

        if niching_method == "crowding":
            population = crowding_replacement(parents, offspring, fitness_function)
        else:
            population = offspring

    return initial_population, population, Fmax_list, Favg_list, Fmin_list


def plot_fitness(Fmax_list, Favg_list, Fmin_list, title, pdf):
    plt.figure()
    plt.plot(Fmax_list, label='Fmax')
    plt.plot(Favg_list, label='Favg')
    plt.plot(Fmin_list, label='Fmin')
    plt.xlabel('Generation')
    plt.ylabel('Fitness')
    plt.legend()
    plt.title(title)
    pdf.savefig()
    plt.close()

def plot_population(initial_population, final_population, fitness_function, title, lower_bound, upper_bound, pdf):
    x = np.linspace(lower_bound, upper_bound, 1000)
    y = [fitness_function([int(b) for b in f"{int(v * (2**NUM_BITS - 1)) :010b}"]) for v in x]

    plt.figure()
    plt.plot(x, y, label='Function Landscape')
    plt.scatter([decode(ind, lower_bound, upper_bound) for ind in initial_population], [fitness_function(ind) for ind in initial_population], color='red', label='Initial Population')
    plt.scatter([decode(ind, lower_bound, upper_bound) for ind in final_population], [fitness_function(ind) for ind in final_population], color='green', label='Final Population')
    plt.xlabel('x')
    plt.ylabel('Fitness')
    plt.title(title)
    plt.legend()
    pdf.savefig()
    plt.close()


def run_experiments_with_niching():
    fitness_functions = [
        (fitness_x_squared, "x^2", 1, 50),
        (fitness_m1, "M1", 0, 1),
        (fitness_m4, "M4", 0, 1)
    ]
    mutation_rates = [0.2, 0.5, 0.8]
    crossover_rates = [0.2, 0.5, 0.8]
    niching_methods = [None, "sharing", "crowding"]

    pdf_path = "GA_Experiments_Niching.pdf"  
    with PdfPages(pdf_path) as pdf:
        for fitness_function, func_name, lb, ub in fitness_functions:
            for Pm in mutation_rates:
                for Pc in crossover_rates:
                    for niching in niching_methods:
                        title_suffix = f"Pm={Pm}, Pc={Pc}, Niching={niching}"
                        initial_pop, final_pop, Fmax_list, Favg_list, Fmin_list = genetic_algorithm(
                            fitness_function, lower_bound=lb, upper_bound=ub, mutation_rate=Pm, crossover_rate=Pc, niching_method=niching
                        )
                        plot_fitness(Fmax_list, Favg_list, Fmin_list, f"{func_name} {title_suffix}", pdf)
                        plot_population(initial_pop, final_pop, fitness_function, f"{func_name} Landscape {title_suffix}", lb, ub, pdf)
    return pdf_path

pdf_file_niching = run_experiments_with_niching()
print(f"Results saved to: {pdf_file_niching}")

Results saved to: GA_Experiments_Niching.pdf


In [43]:
import numpy as np
import random
import math
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

# Parameters
POP_SIZE = 100           
NUM_BITS = 10            
GENERATIONS = 100       
DEFAULT_MUT_RATE = 0.1   
DEFAULT_CROSS_RATE = 0.8 

def decode(binary_string, lower_bound=0, upper_bound=1):
    decimal_value = int("".join(map(str, binary_string)), 2)
    max_decimal = 2**NUM_BITS - 1
    return lower_bound + (decimal_value / max_decimal) * (upper_bound - lower_bound)

# Fitness Functions
def fitness_x_squared(individual):
    x = decode(individual, lower_bound=1, upper_bound=50)  
    return x**2  

def fitness_m1(individual):
    x = decode(individual)  
    return math.sin(5 * math.pi * x)**6

def fitness_m4(individual):
    x = decode(individual)  
    return math.exp(-2 * (math.log(2)) * ((x - 0.08) / 0.854)**2) * math.sin(5 * math.pi * (x**0.75 - 0.05))**6

def initialize_pop():
    return [[random.randint(0, 1) for _ in range(NUM_BITS)] for _ in range(POP_SIZE)]

def selection(population, fitness_function):
    fitness_values = [(ind, fitness_function(ind)) for ind in population]
    sorted_population = sorted(fitness_values, key=lambda x: x[1], reverse=True)
    return [x[0] for x in sorted_population[:POP_SIZE // 2]]

def crossover(parents, crossover_rate=DEFAULT_CROSS_RATE):
    offspring = []
    for _ in range(POP_SIZE):
        parent1 = random.choice(parents)
        parent2 = random.choice(parents)
        if random.random() < crossover_rate:
            crossover_point = random.randint(1, NUM_BITS - 1)
            child = parent1[:crossover_point] + parent2[crossover_point:]
        else:
            child = parent1
        offspring.append(child)
    return offspring

def mutate(offspring, mutation_rate=DEFAULT_MUT_RATE):
    mutated_offspring = []
    for individual in offspring:
        for i in range(len(individual)):
            if random.random() < mutation_rate:
                individual[i] = 1 - individual[i]  
        mutated_offspring.append(individual)
    return mutated_offspring

# Fitness Sharing
def sharing_function(distance, sigma_share, alpha=1):
    if distance < sigma_share:
        return 1 - (distance / sigma_share) ** alpha
    return 0

def apply_fitness_sharing(population, fitness_function, sigma_share=0.1):
    shared_fitness_values = []
    for i, ind_i in enumerate(population):
        niche_count = 0
        for j, ind_j in enumerate(population):
            if i != j:  
                distance = abs(decode(ind_i) - decode(ind_j))  
                niche_count += sharing_function(distance, sigma_share)
        
        original_fitness = fitness_function(ind_i)
        shared_fitness = original_fitness / (1 + niche_count)  
        shared_fitness_values.append(shared_fitness)
    return shared_fitness_values

# Crowding
def crowding_replacement(parents, offspring, fitness_function):
    new_population = []
    for child in offspring:
        parent = min(parents, key=lambda p: abs(decode(p) - decode(child)))  
        if fitness_function(child) > fitness_function(parent):
            new_population.append(child)  
        else:
            new_population.append(parent)  
    return new_population

def genetic_algorithm(fitness_function, lower_bound=0, upper_bound=1, mutation_rate=DEFAULT_MUT_RATE, crossover_rate=DEFAULT_CROSS_RATE, generations=GENERATIONS, niching_method=None):
    population = initialize_pop()
    initial_population = population[:]

    Fmax_list, Favg_list, Fmin_list = [], [], []

    for generation in range(generations):
        if niching_method == "sharing":
            fitness_values = apply_fitness_sharing(population, fitness_function, sigma_share=0.1)
        else:
            fitness_values = [fitness_function(ind) for ind in population]

        Fmax = max(fitness_values)
        Favg = sum(fitness_values) / len(fitness_values)
        Fmin = min(fitness_values)
        Fmax_list.append(Fmax)
        Favg_list.append(Favg)
        Fmin_list.append(Fmin)

        parents = selection(population, fitness_function)
        offspring = crossover(parents, crossover_rate)
        offspring = mutate(offspring, mutation_rate)

        if niching_method == "crowding":
            population = crowding_replacement(parents, offspring, fitness_function)
        else:
            population = offspring

    return initial_population, population, Fmax_list, Favg_list, Fmin_list

def run_multiple_experiments(fitness_function, lower_bound, upper_bound, mutation_rate, crossover_rate, runs=10, generations=GENERATIONS, niching_method=None):
    all_Fmax, all_Favg, all_Fmin = [], [], []

    for _ in range(runs):
        _, _, Fmax_list, Favg_list, Fmin_list = genetic_algorithm(
            fitness_function, lower_bound, upper_bound, mutation_rate, crossover_rate, generations, niching_method
        )
        all_Fmax.append(Fmax_list)
        all_Favg.append(Favg_list)
        all_Fmin.append(Fmin_list)

    avg_Fmax = np.mean(all_Fmax, axis=0)
    avg_Favg = np.mean(all_Favg, axis=0)
    avg_Fmin = np.mean(all_Fmin, axis=0)

    return avg_Fmax, avg_Favg, avg_Fmin

def run_and_save_to_pdf(fitness_functions, mutation_rates, crossover_rates, niching_methods, generations):
    pdf_path = "GA_Results_With_Niching.pdf"
    with PdfPages(pdf_path) as pdf:
        for fitness_function, func_name, lb, ub in fitness_functions:
            for niching in niching_methods:
                for Pm in mutation_rates:
                    for Pc in crossover_rates:
                        # Get average statistics for 10 runs
                        avg_Fmax, avg_Favg, avg_Fmin = run_multiple_experiments(
                            fitness_function, lower_bound=lb, upper_bound=ub,
                            mutation_rate=Pm, crossover_rate=Pc, runs=10, generations=generations, niching_method=niching
                        )
                        
                        # Get one run for landscape
                        initial_pop, final_pop, _, _, _ = genetic_algorithm(
                            fitness_function, lower_bound=lb, upper_bound=ub,
                            mutation_rate=Pm, crossover_rate=Pc, generations=generations, niching_method=niching
                        )
                        
                        # Plot fitness
                        plt.figure(figsize=(6, 6))
                        plt.plot(avg_Fmax, label='Avg Fmax')
                        plt.plot(avg_Favg, label='Avg Favg')
                        plt.plot(avg_Fmin, label='Avg Fmin')
                        plt.title(f"Fitness: {func_name}, Pm={Pm}, Pc={Pc}, Niching={niching}")
                        plt.xlabel('Generation')
                        plt.ylabel('Fitness')
                        plt.legend()
                        pdf.savefig()
                        plt.close()

                        # Plot landscape
                        x = np.linspace(lb, ub, 1000)
                        y = [fitness_function([int(b) for b in f"{int(v * (2**NUM_BITS - 1)) :010b}"]) for v in x]
                        plt.figure(figsize=(6, 6))
                        plt.plot(x, y, label='Function Landscape')
                        plt.scatter([decode(ind, lb, ub) for ind in initial_pop],
                                    [fitness_function(ind) for ind in initial_pop], color='red', label='Initial Population')
                        plt.scatter([decode(ind, lb, ub) for ind in final_pop],
                                    [fitness_function(ind) for ind in final_pop], color='green', label='Final Population')
                        plt.title(f"Landscape: {func_name}, Pm={Pm}, Pc={Pc}, Niching={niching}")
                        plt.xlabel('x')
                        plt.ylabel('Fitness')
                        plt.legend()
                        pdf.savefig()
                        plt.close()

    return pdf_path

# Configurations
fitness_functions = [
    (fitness_x_squared, "x^2", 1, 50),
    (fitness_m1, "M1", 0, 1),
    (fitness_m4, "M4", 0, 1)
]

mutation_rates = [0.2, 0.5, 0.8]
crossover_rates = [0.2, 0.5, 0.8]
niching_methods = [None, "sharing", "crowding"]



# Run experiments and save to PDF
pdf_file = run_and_save_to_pdf(fitness_functions, mutation_rates, crossover_rates, niching_methods, generations=GENERATIONS)
print(f"Results saved to: {pdf_file}")


Results saved to: GA_Results_With_Niching.pdf
